### Imports

In [57]:
# %pip install -r dependencies.txt

In [73]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import requests
import scipy
import importlib
import helper, consts
importlib.reload(consts)
importlib.reload(helper)

<module 'helper' from '/Users/mike/Documents/School/Fa23/Clinic/code/helper.py'>

### CONSTANTS

In [69]:
ROW = consts.ROW
COL = consts.COL
CLEANED_DATA_PATH = consts.CLEANED_DATA_PATH
DATA_PATH = consts.RAW_DATA_PATH
RESPONSE_NAME = consts.RESPONSE_NAME

TRAIN_START_DATE = "20150101"
TRAIN_END_DATE = "20150601" # Up to but not including
TEST_START_DATE = "20150701"
TEST_END_DATE = "20150801"

In [70]:
REGRESSION_TYPES = helper.Regression('OLS').list_all_regression_types()
REGRESSION_TYPES

1: OLS
2: LASSO
3: XGBOOST


### Read data

In [78]:
x_cols = ["rrirpnxm_nt_0", "rrirpnxm_lst15_0","rrirpnxm_lsthrx15_0", "rrirpnxm_toxhr_0", "relvol_nt_0"]
train_df = helper.get_df(TRAIN_START_DATE, TRAIN_END_DATE, x_cols)

### Interaction Terms

In [62]:
# interactingTerms = [["relvol_nt_0", col] for col in x_cols[:-1]]
# interactingTerms

In [63]:
# interactingTerms_df = helper.get_df_with_interaction_terms(train_df, interactingTerms)

In [64]:
# interactingTerms_df.head()

In [65]:
# rosy: making my own interacting terms

# col_pairs = [
#     ['relvol_nt_0', 'rrirpnxm_nt_0'],
#     ["relvol_lst15_0", "rrirpnxm_lst15_0"],
#     ["relvol_toxhr_0", "rrirpnxm_toxhr_0"],
#     ["relvol_lsthrx15_0", "rrirpnxm_lsthrx15_0"],
# ]
# interaction_terms_train_df = helper.get_df_with_interaction_terms(train_df, col_pairs)
# interaction_terms_test_df = helper.get_df_with_interaction_terms(test_df, col_pairs)

### Transform the data if needed

#### Ordinary Least Squares

In [79]:
ols_regression_model = helper.Regression('OLS')
model_attributes = ols_regression_model.train(train_df, RESPONSE_NAME)
model_attributes

array([3.86475959e-02, 4.68771384e-03, 1.99013644e-03, 1.70877863e-03,
       3.94435165e-05, 3.85589377e-05])

In [80]:
ols_regression_model.get_average_metrics(TEST_START_DATE, TEST_END_DATE, x_cols)

/usr/local/lib/python3.10/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(


ValueError: X has 6 features, but LinearRegression is expecting 1 features as input.

#### OLS w/ Interacting Terms

In [ ]:
ols_interacting_model = helper.Regression('OLS')
ols_interacting_model.execute(interactingTerms_df, RESPONSE_NAME, interactingTerms_df)
ols_interacting_model.get_metric()

#### LASSO

In [ ]:
lasso_model = helper.Regression('LASSO')
lasso_model.execute(train_df, RESPONSE_NAME, test_df)
lasso_model.get_metric()

### XGBoost



In [ ]:
xgb_model = helper.Regression('XGboost')
xgb_model.execute(train_df, RESPONSE_NAME, test_df)
xgb_model.get_metric()